# **Fera Formidável 4.13**

* Caio M. C. Ruas - RM: 24010

## Introdução

O objetivo dessa fera é, dado uma base de dados com elementos e seus custos, encontrar uma liga de três elementos que tenha o maior custo possível. Essa liga ternária deve ser da forma xA.y B.z C, onde x, y e z são as quantidades dos elementos A, B e C, respectivamente, e a soma de x, y e z deve ser igual a 100 g. Além disso, cada elemento deve ter uma quantidade mínima de 5 g.

Para atender esse objetivo, utilizaremos um algoritmo genético. A ideia é que cada indivíduo da população represente uma liga ternária, e o algoritmo irá evoluir essa população ao longo de várias gerações. Para atender às restrições, ao invés de penalizar no fitness, vamos garantir que as ligas geradas atendam às restrições de quantidade mínima e soma total.

## Bibliotecas e definições iniciais

Nessa célula, importamos as bibliotecas necessárias e definimos a base de dados com os elementos e seus custos. Também definimos algumas variáveis que delimitam as condições do problema, como a quantidade mínima de cada elemento e o máximo da liga.

In [1]:
import random
import numpy as np

random.seed(7)
np.random.seed(7)

preco = {
    "H": 1.39, "He": 24, "Li": 85.6, "Be": 857, "B": 3.68, "C": 0.122,
    "N": 0.14, "O": 0.154, "F": 2.16, "Ne": 240, "Na": 3.43, "Mg": 2.32,
    "Al": 1.79, "Si": 1.7, "P": 2.69, "S": 0.0926, "Cl": 0.082, "Ar": 0.931,
    "K": 13.6, "Ca": 2.35, "Sc": 3460, "Ti": 11.7, "V": 385, "Cr": 9.4,
    "Mn": 1.82, "Fe": 0.424, "Co": 32.8, "Ni": 13.9, "Cu": 6, "Zn": 2.55,
    "Ga": 148, "Ge": 1010, "As": 1.31, "Se": 21.4, "Br": 4.39, "Kr": 290,
    "Rb": 15500, "Sr": 6.68, "Y": 31, "Nb": 85.6, "Mo": 40.1, "Tc": 100000,
    "Ru": 10600, "Rh": 147000, "Pd": 49500, "Ag": 521, "Cd": 2.73, "In": 167,
    "Sn": 18.7, "Sb": 5.79, "Te": 63.5, "I": 35, "Xe": 1800, "Cs": 61800,
    "Ba": 0.275, "La": 4.92, "Ce": 4.71, "Pr": 103, "Nd": 57.5, "Pm": 460000,
    "Sm": 13.9, "Eu": 31.4, "Gd": 28.6, "Tb": 658, "Dy": 307, "Ho": 57.1,
    "Er": 26.4, "Tm": 3000, "Yb": 17.1, "Lu": 643, "Hf": 900, "Ta": 312,
    "W": 35.3, "Re": 4150, "Os": 12000, "Ir": 56200, "Pt": 27800, "Hg": 30.2,
    "Tl": 4200, "Pb": 2, "Bi": 6.36, "Po": 49200000000000, "Ac": 29000000000000,
    "Th": 287, "Pa": 280000, "U": 101, "Np": 660000, "Pu": 6490000,
    "Am": 750000, "Cm": 160000000000, "Bk": 185000000000, "Cf": 185000000000
}

TODOS_ELEMENTOS_DISPONIVEIS = list(preco.keys())
NUM_ELEMENTOS_LIGA = 3
MASSA_TOTAL_G = 100.0
MASSA_MINIMA_POR_ELEMENTO_G = 5.0

## Funções auxiliares

Aqui, definimos as funções auxiliares para o problema: Uma função responsável por garantir que as massas sejam válidas e as funções que geram indivíduos e populações iniciais. Essas funções são essenciais para o funcionamento do algoritmo genético. Inicialmente, foi tentando acoplar a restrição de massa diretamente na geração de indivíduos, mas não funcinou muito bem, pareceu muito mais claro e versátil deixar a geração de indivíduos e a validação de massa separadas.

In [2]:
# --- FUNÇÕES AUXILIARES ---

def _gerar_massas_validas():
    """
    Função auxiliar para gerar 3 massas que somam 100g, com cada uma sendo >= 5g..
    """
    massa_restante = MASSA_TOTAL_G - NUM_ELEMENTOS_LIGA * MASSA_MINIMA_POR_ELEMENTO_G
    
    ponto1 = random.uniform(0, massa_restante)
    ponto2 = random.uniform(0, massa_restante)
    
    cortes = sorted([0, ponto1, ponto2, massa_restante])
    
    massa_adicional_1 = cortes[1] - cortes[0]
    massa_adicional_2 = cortes[2] - cortes[1]
    massa_adicional_3 = cortes[3] - cortes[2]
    
    massas = [
        round(MASSA_MINIMA_POR_ELEMENTO_G + massa_adicional_1, 2),
        round(MASSA_MINIMA_POR_ELEMENTO_G + massa_adicional_2, 2),
        round(MASSA_MINIMA_POR_ELEMENTO_G + massa_adicional_3, 2),
    ]
    
    massas[0] += MASSA_TOTAL_G - sum(massas)
    
    random.shuffle(massas)
    return massas

def cria_candidato_liga():
    """
    Cria um candidato para a liga garantindo elementos únicos e massas válidas.
    """
    elementos = random.sample(TODOS_ELEMENTOS_DISPONIVEIS, NUM_ELEMENTOS_LIGA)
    
    massas = _gerar_massas_validas()
    
    candidato = list(zip(elementos, massas))
    return candidato

def cria_populacao_liga(tamanho_populacao):
    """
    Cria uma população inicial de candidatos válidos.)
    """
    return [cria_candidato_liga() for _ in range(tamanho_populacao)]

## Funções do algoritmo genético

Aqui, definimos as funções necessárias para o algoritmo genético, incluindo a função de fitness, que avalia o custo da liga, e as funções de seleção, cruzamento e mutação.

Neste caso, optamos por utilizar uma seleção por roleta, onde indivíduos com maior custo têm mais chances de serem selecionados. Optamos por isso pois entendemos que a seleção por torneio poderia levar a uma convergência prematura ou a perda de diversidade genética, já que o custo é um fator determinante para a qualidade da liga.

A função de cruzamento foi feita de forma que os filhos herdem elementos aleatórios dentre as possibilidades dos dois pais.

A função de mutação foi feita de forma que é possível trocar um elemento por outro aleatório dentre todas as possibilidades originais ou é possível mutar a quantidade de todos os elementos, sem mudá-los.

É importante ressaltar que todas as funções de cruzamento e mutação garantem que os filhos gerados atendam às restrições de quantidade mínima e soma total.

In [3]:
# --- FUNÇÕES DE AVALIAÇÃO E SELEÇÃO ---

def avalia_candidato_liga(candidato):
    """função de avaliação"""
    valor_total = 0
    for elemento, massa_g in candidato:
        if elemento not in preco:
            continue 
        preco_por_kg = preco[elemento]
        massa_kg = massa_g / 1000.0
        valor_total += preco_por_kg * massa_kg
    return valor_total

def selecao_roleta_max(populacao, fitness):
    """função de seleção"""
    min_fitness = min(fitness)

    adjusted_fitness = [f - min_fitness + 1 for f in fitness]
    total_adjusted_fitness = sum(adjusted_fitness)
    if total_adjusted_fitness == 0:
        return random.choice(populacao).copy()
    probabilidades = [f / total_adjusted_fitness for f in adjusted_fitness]
    indice_selecionado = np.random.choice(len(populacao), p=probabilidades)
    return populacao[indice_selecionado].copy() 

# --- FUNÇÕES DE CRUZAMENTO E MUTAÇÃO ---

def cruzamento_liga(pai1, pai2, taxa_cruzamento=0.7):
    """
    Realiza o cruzamento de ponto simples de forma a garantir sempre um filho válido.
    """
    filho1, filho2 = pai1.copy(), pai2.copy()

    if random.random() < taxa_cruzamento:

        elementos_pai1 = [gene[0] for gene in pai1]
        elementos_pai2 = [gene[0] for gene in pai2]
        
        pool_de_elementos = sorted(list(set(elementos_pai1 + elementos_pai2)))
        
        while len(pool_de_elementos) < NUM_ELEMENTOS_LIGA:
            novo_elemento = random.choice(TODOS_ELEMENTOS_DISPONIVEIS)
            if novo_elemento not in pool_de_elementos:
                pool_de_elementos.append(novo_elemento)
        
        elementos_filho1 = random.sample(pool_de_elementos, NUM_ELEMENTOS_LIGA)
        elementos_filho2 = random.sample(pool_de_elementos, NUM_ELEMENTOS_LIGA)
        
        massas_filho1 = _gerar_massas_validas()
        massas_filho2 = _gerar_massas_validas()
        
        filho1 = list(zip(elementos_filho1, massas_filho1))
        filho2 = list(zip(elementos_filho2, massas_filho2))

    return filho1, filho2

def mutacao_liga(candidato, taxa_mutacao=0.1):
    """
    Realiza a mutação de forma a garantir sempre um candidato válido.
    Temos duas possíveis mutações:
    1. Trocar um elemento.
    2. Reajustar as massas.
    """
    if random.random() < taxa_mutacao:

        if random.random() < 0.5:
            indice_mutacao = random.randint(0, len(candidato) - 1)
            elementos_atuais = [gene[0] for gene in candidato]
            
            novo_elemento = random.choice(TODOS_ELEMENTOS_DISPONIVEIS)
            while novo_elemento in elementos_atuais:
                novo_elemento = random.choice(TODOS_ELEMENTOS_DISPONIVEIS)
            
            candidato[indice_mutacao] = (novo_elemento, candidato[indice_mutacao][1])
        
        else:
            elementos_atuais = [gene[0] for gene in candidato]
            novas_massas = _gerar_massas_validas()
            candidato = list(zip(elementos_atuais, novas_massas))
            
    return candidato

## Aplicando o algoritmo genético

Nessa etapa, juntamos as funções definidas anteriormente para estruturar e aplicar a função do algoritmo genético em sí. Então, definimos os parâmetros do algoritmo, como a população inicial, o número de gerações e a taxa de mutação. Em seguida, aplicamos o algoritmo e exibimos os resultados.

Primeiro, vamos estruturar o algoritmo. É válido ressaltar que o algoritmo é estruturado usando a ideia de elitismo, então o melhor indivíduo da geração anterior é sempre mantido na próxima geração.

In [4]:
def algoritmo_genetico_liga(tamanho_populacao, num_geracoes, taxa_mutacao=0.1, taxa_cruzamento=0.7):
    """
    Executa o algoritmo genético para otimizar a liga.
    """
    populacao = cria_populacao_liga(tamanho_populacao)
    melhor_candidato_global = None
    melhor_fitness_global = float('-inf')

    for geracao in range(num_geracoes):

        fitness_populacao = [avalia_candidato_liga(ind) for ind in populacao]

        melhor_fitness_geracao = max(fitness_populacao)
        indice_melhor_geracao = fitness_populacao.index(melhor_fitness_geracao)
        melhor_candidato_geracao = populacao[indice_melhor_geracao]

        if melhor_fitness_geracao > melhor_fitness_global:
            melhor_fitness_global = melhor_fitness_geracao
            melhor_candidato_global = melhor_candidato_geracao.copy()
            print(f"Geração {geracao + 1}: Novo melhor fitness = ${melhor_fitness_global:,.2f}")

        nova_populacao = []
        
        nova_populacao.append(melhor_candidato_geracao.copy())

        while len(nova_populacao) < tamanho_populacao:
            pai1 = selecao_roleta_max(populacao, fitness_populacao)
            pai2 = selecao_roleta_max(populacao, fitness_populacao)
            
            filho1, filho2 = cruzamento_liga(pai1, pai2, taxa_cruzamento)
            
            filho1 = mutacao_liga(filho1, taxa_mutacao)
            filho2 = mutacao_liga(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            
            if len(nova_populacao) < tamanho_populacao:
                nova_populacao.append(filho2)
        
        populacao = nova_populacao

    return melhor_candidato_global, melhor_fitness_global

Com a função do algoritmo genético definida, podemos aplicá-la para encontrar a melhor liga possível. 

Faremos isso agora.

Usaremos uma população de 50 indivíduos, por 500 gerações. Decidi aumentar a taxa de mutação para 0.3, pois acredito que isso pode ajudar a explorar mais o espaço de soluções e evitar a convergência prematura. A taxa de cruzamento será de 0.7.

In [5]:
TAMANHO_POPULACAO = 50
NUM_GERACOES = 500

melhor_candidato, melhor_fitness = algoritmo_genetico_liga(
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES,
    taxa_mutacao=0.3,
    taxa_cruzamento=0.7
)

print("\n--- Resultado Final ---")
print("Melhor liga encontrada:")

massa_total = 0
for elemento, massa in melhor_candidato:
    print(f"  - Elemento: {elemento}, Massa: {massa:.2f}g")
    massa_total += massa
print(f"Massa Total: {massa_total:.2f}g")
print(f"\nCusto (Fitness) da Liga: ${melhor_fitness:,.2f}")

Geração 1: Novo melhor fitness = $1,909,452,002,242.37
Geração 2: Novo melhor fitness = $3,097,562,000,000.04
Geração 3: Novo melhor fitness = $3,474,996,000,589.07
Geração 4: Novo melhor fitness = $4,200,786,000,342.54
Geração 12: Novo melhor fitness = $4,227,078,000,060.96
Geração 15: Novo melhor fitness = $4,266,866,000,071.16
Geração 21: Novo melhor fitness = $4,301,300,000,000.31
Geração 23: Novo melhor fitness = $4,365,006,000,000.01
Geração 32: Novo melhor fitness = $4,423,644,000,344.52
Geração 39: Novo melhor fitness = $4,457,700,005,347.50
Geração 45: Novo melhor fitness = $4,465,172,000,000.00
Geração 90: Novo melhor fitness = $4,551,210,000,000.00
Geração 93: Novo melhor fitness = $4,551,210,000,000.02
Geração 115: Novo melhor fitness = $4,551,210,000,000.20
Geração 142: Novo melhor fitness = $4,551,210,000,077.81
Geração 152: Novo melhor fitness = $4,551,210,000,502.00
Geração 328: Novo melhor fitness = $4,551,210,001,405.60
Geração 437: Novo melhor fitness = $4,552,138,70

Perfeito! Podemos verificar que o algoritmo convergiu para uma liga com custo $4,561,549,500,000.00, sendo que seus elementos são:

  - Elemento: Po, Massa: 89.53g
  - Elemento: Cf, Massa: 5.10g
  - Elemento: Ac, Massa: 5.37g

No caso desse problema, que tem um único objetivo direto, podemos verificar por cálculos simples se realmente é essa a melhor solução! A ideia é simples, como não há nenhuma restrição além das massas mínimas e nenhum outro objetivo, podemos simplesmente escolher os três elementos com maior custo e formar uma liga com eles!

Com os elementos de maior custo em mão, podemos pensar o seguinte: A liga de maior custo possível, teria 90g do elemento mais caro, 5g do segundo mais caro e 5g do terceiro mais caro. Assim, podemos calcular o custo dessa liga e verificar se é próximo ao que obtivemos com o algoritmo genético.

In [6]:
preco_copia = preco.copy()

elementos_mais_caros = []

for _ in range(3):
    max_element = max(preco_copia.values())
    max_elements = [elemento for elemento, valor in preco_copia.items() if valor == max_element]
    for elemento in max_elements:
        print(f"  - Elemento: {elemento}, Custo: ${max_element:.2f}")
    preco_copia.pop(max_elements[0], None) 
    elementos_mais_caros.append((max_elements[0], max_element))

massa_mais_caro = 90.0
massa_outros = 5.0

custo_liga_maxima = (
    massa_mais_caro / 1000.0 * elementos_mais_caros[0][1] +
    massa_outros / 1000.0 * elementos_mais_caros[1][1] +
    massa_outros / 1000.0 * elementos_mais_caros[2][1]
)

print("\n--- Cálculo da Liga Máxima ---")
print(f"\nCusto da Liga Máxima: ${custo_liga_maxima:.2f}")

print(f"Custo da Liga Encontrada: ${melhor_fitness:.2f}")


  - Elemento: Po, Custo: $49200000000000.00
  - Elemento: Ac, Custo: $29000000000000.00
  - Elemento: Bk, Custo: $185000000000.00
  - Elemento: Cf, Custo: $185000000000.00

--- Cálculo da Liga Máxima ---

Custo da Liga Máxima: $4573925000000.00
Custo da Liga Encontrada: $4561549500000.00


Podemos perceber que o custo da nossa liga está bem próximo ao custo máximo possível, inclusive, os elementos mais caros são os mesmos que o algoritmo genético encontrou, além disso, a proporção entre os elementos que o algoritmo encontrou é bem próximo da proporção ideal que teorizamos. 

## Conclusão

O algoritmo genético foi capaz de encontrar uma liga ternária com custo muito próximo do máximo possível, atendendo às restrições de quantidade mínima e soma total. A solução encontrada é eficiente e demonstra a eficácia do algoritmo genético para problemas de otimização como este. Apesar de não ter sido a solução exata, o resultado é satisfatório e mostra que o algoritmo é capaz de explorar o espaço de soluções de forma eficaz, talvez com mais gerações ou uma população maior, poderíamos ter chegado ainda mais próximo do custo máximo possível.

## Referências

No geral, não foram utilizadas referências externas para a implementação deste monstrinho, mas é importante ressaltar que o material de aula foi fundamental para a compreensão do problema e das abordagens de otimização.